In [76]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error as mse
from joblib import dump,load

In [77]:
df=pd.read_csv("data//clean//vehiculos.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14631 entries, 0 to 14657
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Marca                   14631 non-null  object 
 1   Modelo                  14631 non-null  object 
 2   Precio                  14631 non-null  float64
 3   Potencia                14631 non-null  float64
 4   Tipo vendedor           14631 non-null  object 
 5   Categoría               14631 non-null  object 
 6   Tipo de vehículo        14631 non-null  object 
 7   puertas                 14631 non-null  int64  
 8   Versión del país        14631 non-null  object 
 9   Garantía                14631 non-null  int64  
 10  Kilometraje             14631 non-null  float64
 11  Año                     14631 non-null  int64  
 12  Tipo de cambio          14631 non-null  object 
 13  Capacidad               14631 non-null  float64
 14  Consumo de combustible  14631 non-null  flo

Como 'Versión del país' siempre contiene el valor 'España' no es relevante y lo podemos eliminar.

In [78]:
df=df.drop('Versión del país', axis=1)

Definimos las columnas numéricas y las categóricas:

In [59]:
num_cols=df.select_dtypes(['int64', 'float64']).columns
cat_cols=df.select_dtypes(['object']).columns

De las columnas numéricas hay que mirar cuales tienen distribución gausiana, para saber a cuales podemos estandarizar con el StandartScaler.

In [60]:
def get_gaussian_columns(df):
    gaus_cols=[]
    for column in df.columns:
        stat, p = shapiro(df[column])
        if p > 0.05:
            gaus_cols.append(column)
    return gaus_cols

In [61]:
df_num=df[num_cols]
gaus_cols=get_gaussian_columns(df_num)
print(gaus_cols)

[]


c:\Users\mabi1\anaconda3\envs\proj\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Ninguna es probable que tenga distribución gausiana. Ahora de las columnas numéricas vamos a separar las que tienen outliers de las que no, para ello cogeremos el dataframe con solo las columnas numéricas, tomaremos también el primer cuantil, el tercero y la diferencia entre ambos y con una fórmula matemática seleccionaremos las columnas donde hay outliers. Una vez obtenidas las columnas con outliers haremos un drop de las mismas en el dataframe de columnas numéricas y las restantes serán las que no tienen outliers. Es importante tener en cuenta cuales tienen outliers y cuales no para saber qué tipo de escalado aplicarles.

In [62]:
q1 = df_num.quantile(0.25)
q3 = df_num.quantile(0.75)
diff = q3 - q1
out_cols=df_num.columns[((df_num < (q1 - 1.5 * diff)) |(df_num > (q3 + 1.5 * diff))).any(axis=0)]
#Conseguimos las columnas sin outliers eliminando las que tienen outliers del conjunto de columnas numéricas
no_out_cols=df_num.drop(out_cols, axis=1).columns
#Eliminamos la variable target
out_cols=out_cols.drop('Precio')

In [63]:
print('Numéricas con outliers',list(out_cols))
print('Numéricas sin outliers',list(no_out_cols))
print('Categóricas', list(cat_cols))

Numéricas con outliers ['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad', 'Consumo de combustible', 'plazas', 'Número de marchas', 'Número de cilindros', 'Peso']
Numéricas sin outliers ['Mes', 'CP']
Categóricas ['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo', 'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción', 'Tipo de combustible', 'Ciudad', 'provincia']


Así pues, ahora tenemos las columnas de nuestro dataset divididas en tres sets: columnas categóricas (cat_cols), columnas numéricas con outliers(out_cols) y columnas numéricas sin outliers (no_out_cols).

A todas las columnas le añadiremos algún tipos de SimpleImputer, no para los NaN que había, que ya se han limpiado, sino por los que podrían llegar con los nuevos datos a predecir.

A las columnas categóricas les aplicaremos OneHotEncoder y posteriormente BinaryEncoder, para poder inlcuirlas en el PCA, a las que tienen outliers RobustScaler y a las que no tienen outliers MinMaxScaler.

In [64]:
out_pipeline = Pipeline(steps=[
    ('imputer_num', SimpleImputer(strategy = 'median')),
    ('scale',RobustScaler())
])

no_out_pipeline = Pipeline(steps=[
    ('imputer_num', SimpleImputer(strategy = 'median')),
    ('scale',MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('imputer_num', SimpleImputer(strategy = 'most_frequent')),
    ('scale', OneHotEncoder(handle_unknown='ignore', sparse=False, drop='first'))
])

Añadiremos estas pequeñas pipeslines a un ColumnTransformer para que a cada tipo de columna se le aplique lo indicado. Le indicaremos remainder='drop' para que ignore cualquier columna del dataframe que no esté contemplada en esas categorías. Y n_jobs= -1 para que use todos los procesadores que tengamos en paralelo.

In [65]:
preprocessor = ColumnTransformer(transformers=[
    ('out_pipeline',out_pipeline,out_cols),
    ('no_out_pipeline',no_out_pipeline,no_out_cols),
    ('cat_pipeline', cat_pipeline, cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

Vamos a visualizar la pipeline

In [66]:
set_config(display='diagram')
display(preprocessor)

ColumnTransformer(n_jobs=-1,
                  transformers=[('out_pipeline',
                                 Pipeline(steps=[('imputer_num',
                                                  SimpleImputer(strategy='median')),
                                                 ('scale', RobustScaler())]),
                                 Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')),
                                ('no_out_pipeline',
                                 Pipeline(steps=...
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scale',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
                                 Index(['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo',
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])

Separamos las variables independientes del target

In [67]:
X=df.drop('Precio', axis=1)
y=df['Precio']

Vamos a aplicarle el preprocesado a las variables independientes

In [68]:
X=preprocessor.fit_transform(X)

In [69]:
X.shape

(14631, 2922)

Al  tener casi 3000 columnas los entrenamientos de modelos llevan casi 1 hora, lo cual no es asumible si queremos probar varios modelos con varios parámetros, así que aplicaremos un PCA y nos quedaremos con el 92% de la varianza, para agilizar el proceso.

Volvemos a asignar X porque en la siguiente pipeline se volverá a preprocesar

In [70]:
X=df.drop('Precio', axis=1)

In [71]:
pca_pipe=Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.94))
])
pca_pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('out_pipeline',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   RobustScaler())]),
                                                  Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')),
                                                 ('...
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scale',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo',
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                ('pca', PCA(n_components=0.94))])

In [72]:
#X=pca_pipe.fit_transform(X)
#X.shape

Buscaremos los mejores modelos con los parámetros por defecto para después tunear los hiperparámetros de los mejores.

In [73]:
lr=LinearRegression()
ridge=Ridge(random_state=101)
enet=ElasticNet(random_state=101)
rfor=RandomForestRegressor(random_state=101)
gb=GradientBoostingRegressor(random_state=101)
sgd=SGDRegressor()
svr=SVR()
bridge=BayesianRidge()
kridge=KernelRidge()

In [74]:
models=[lr,ridge,enet, rfor,gb, sgd, svr, bridge, kridge]
score_means=[]
mse=[]
performance=pd.DataFrame({'model':['Linear','Ridge','ElasticNet','Random Forest','Gradient Boosting','Stochastic Gradient Descent', 'Support Vector Machines', 'Bayesian Ridge', 'Kernel Ridge']})

In [75]:
for model in models:
    pipeline = Pipeline(
        [
            ('preprocessing', preprocessor),
            ('pca', PCA(0.94)),
            ('model', model)
        ]
    )
    score_means.append(cross_val_score(pipeline, X, y, scoring='r2', cv=5).mean())

KeyboardInterrupt: 

In [ ]:
performance['r2_mean']=score_means
performance

,model,r2_mean
0,Linear,-0.481175
1,Ridge,-0.481035
2,ElasticNet,-0.199378
3,Random Forest,-0.269601
4,Gradient Boosting,-0.124199
5,Stochastic Gradient Descent,-0.389500
6,Support Vector Machines,-0.066629
7,Bayesian Ridge,-0.468841
8,Kernel Ridge,-0.635918


De momento los valores para los modelos por defecto son bastante bajos, aún así filtraremos los 4 con mejores puntuaciones para tunear hiperparámetros e intentar obtener puntuaciones mejores. 

Estos serán Support Vector Machines , ElasticNet , Gradient Boosting  y Random Forest.

In [ ]:
params_SVM={
    'model__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'model__shrinking': [True, False]
}
pipe_SVM = Pipeline(
        [
            ('preprocessing', preprocessor),
            ('pca', PCA(0.94)),
            ('model', SVR())
        ])

In [ ]:
gs_SVM = GridSearchCV(pipe_SVM, params_SVM, cv = 5, scoring = ['r2', 'neg_mean_squared_error'], refit='r2', n_jobs = -1)
gs_SVM.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=-1,
                                                          transformers=[('out_pipeline',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scale',
                                                                                          RobustScaler())]),
                                                                         Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros...
                                                                         Index(['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo',
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                                       ('pca', PCA(n_components=0.94)),
                                       ('model', SVR())]),
             n_jobs=-1,
             param_grid={'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'model__shrinking': [True, False]},
             refit='r2', scoring=['r2', 'neg_mean_squared_error'])

In [ ]:
print(f'Best R2: {gs_SVM.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_SVM.best_params_}\n')

Best R2: -0.017

Best parameter set: {'model__kernel': 'linear', 'model__shrinking': True}



In [ ]:
params_GB={
    'model__loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'model__criterion': ['friedman_mse', 'squared_error'],
    'model__random_state' : [101],
    'model__warm_start' : [True, False]
}
pipe_GB= Pipeline(
        [
            ('preprocessing', preprocessor),
            ('pca', PCA(0.94)),
            ('model', GradientBoostingRegressor())
        ])

In [ ]:
gs_GB = GridSearchCV(pipe_GB, params_GB, cv = 5, scoring = ['r2', 'neg_mean_squared_error'], refit='r2', n_jobs = -1)
gs_GB.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=-1,
                                                          transformers=[('out_pipeline',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scale',
                                                                                          RobustScaler())]),
                                                                         Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros...
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                                       ('pca', PCA(n_components=0.94)),
                                       ('model', GradientBoostingRegressor())]),
             n_jobs=-1,
             param_grid={'model__criterion': ['friedman_mse', 'squared_error'],
                         'model__loss': ['squared_error', 'absolute_error',
                                         'huber', 'quantile'],
                         'model__random_state': [101],
                         'model__warm_start': [True, False]},
             refit='r2', scoring=['r2', 'neg_mean_squared_error'])

In [ ]:
print(f'Best R2: {gs_GB.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_GB.best_params_}\n')

Best R2: 0.383

Best parameter set: {'model__criterion': 'friedman_mse', 'model__loss': 'huber', 'model__random_state': 101, 'model__warm_start': True}



In [ ]:
params_enet={
    'model__fit_intercept' : [True, False],
    'model__copy_X' : [True, False],
    'model__warm_start' : [True, False],
    'model__positive' : [True, False],
    'model__random_state' : [101],
    'model__selection' : ['cyclic', 'random']
}
pipe_enet = Pipeline(
        [
            ('preprocessing', preprocessor),
            ('pca', PCA(0.94)),
            ('model', ElasticNet())
        ])

In [ ]:
gs_enet = GridSearchCV(pipe_enet, params_enet, cv = 5, scoring = 'r2', n_jobs = -1)
gs_enet.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=-1,
                                                          transformers=[('out_pipeline',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scale',
                                                                                          RobustScaler())]),
                                                                         Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros...
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                                       ('pca', PCA(n_components=0.94)),
                                       ('model', ElasticNet())]),
             n_jobs=-1,
             param_grid={'model__copy_X': [True, False],
                         'model__fit_intercept': [True, False],
                         'model__positive': [True, False],
                         'model__random_state': [101],
                         'model__selection': ['cyclic', 'random'],
                         'model__warm_start': [True, False]},
             scoring='r2')

In [ ]:
print(f'Best R2: {gs_enet.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_enet.best_params_}\n')

Best R2: 0.160

Best parameter set: {'model__copy_X': True, 'model__fit_intercept': True, 'model__positive': True, 'model__random_state': 101, 'model__selection': 'cyclic', 'model__warm_start': True}



In [ ]:
params_RF={
    'model__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    'model__oob_score' : [True, False],
    'model__random_state' : [101],
    'model__warm_start' : [True, False] 
}
pipe_RF = Pipeline(
        [
            ('preprocessing', preprocessor),
            ('pca', PCA(0.94)),
            ('model', RandomForestRegressor())
        ])

In [ ]:
gs_RF = GridSearchCV(pipe_RF, params_RF, cv = 5, scoring = 'r2', n_jobs = -1)
gs_RF.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=-1,
                                                          transformers=[('out_pipeline',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scale',
                                                                                          RobustScaler())]),
                                                                         Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros...
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                                       ('pca', PCA(n_components=0.94)),
                                       ('model', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'model__criterion': ['squared_error', 'absolute_error',
                                              'friedman_mse', 'poisson'],
                         'model__oob_score': [True, False],
                         'model__random_state': [101],
                         'model__warm_start': [True, False]},
             scoring='r2')

In [ ]:
print(f'Best R2: {gs_RF.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_RF.best_params_}\n')

Best R2: 0.902

Best parameter set: {'model__criterion': 'poisson', 'model__oob_score': True, 'model__random_state': 101, 'model__warm_start': True}



Finalmente el modelo que nos ha dado mejores resultados es el Random Forest Regressor con un R2 de 0.89. También ha sido el que más ha tardado en entrenarse con diferencia. Ahora lo añadiremos a la pipeline final

In [ ]:
best_model=gs_RF.best_estimator_

Guardaremos el dataframe y el modelo para usarlo con streamlit en el archivo 'web.py'

In [ ]:
dump(df, "vehiculos.df")
dump(best_model, "model.joblib")

['model.joblib']

Vamos a importar el modelo para no tener que reentrenarlo (lo cual tarda 90 minutos) después de haber apagado el ordenador y a intentar predecir datos nuevos. No debería haber un load en el mismo documento, es solo para ahorrarnos el tiempo de reentreno si hemos apagado el ordenador.

In [ ]:
df=load('vehiculos.df')
model=load('model.joblib')

In [ ]:
input_data = pd.DataFrame([['Cupra', 'Formentor', '150.0', 'Prof.', 'SUV/4x4/Pickup', 'Ocasión', 5, 12, 26141.0, 2021, 'Manual', 1329.0, 4.8, 'Negro', 'Negro Midnight (metalizado)',
                                'Tracción delantera	', 4.0, 6.0, 4.0, 1420.0, 'Gasolina', 5, 'Zaragoza', 50014, 'Zaragoza']],
                               columns=df.drop('Precio', axis=1).columns)

In [ ]:
predict = model.predict(input_data)[0]

In [ ]:
print(predict)

33287.91
